# Model neuron simulation
## An analysis of learning performance changes in spiking neural networks(SNN)
### 김용주, 김태호

* HH model cell을 실험 조건에 맞게 배치해 볼 것
    * Ball And Stick Tutorial에서는 z축을 중심으로 2Pi/n rad마다 배치하였음.
    * 해당 실험에서는 n by n의 excitatory neurons를 z=0에 배치, n by n (n=10, 15, 20)의 inhibitory neurons를 z=-1에 배치해 보고자 함
    * excitatory neuron -> inhibitory neurons는 x 좌표와 y좌표가 둘 다 맞을 때 연결
    * inhibitory neuron -> excitatory neurons는 x 좌표와 y좌표가 둘 다 맞을 때를 제외하고 연결

* Q1 : input stimulation은 어떻게 구현할 수 있을까?
    * Convolutional한 MNIST 데이터셋으로 어떻게 excitatory neurons을 흥분시킬 수 있는가? -> NetCon을 이용하여 Value 에서 Frequency로 바꾸는 작업이 필요할 듯
* Q2 : 신경망의 가중치 설정 및 조작
    * CNN perceptron -> SNN neuron
        * Weight -> Synaptic Weight (Weight term)
        * Transfer function 에 대응되는 개념이 SNN에는 존재하지 않는가?
        * Refractory는 불변의 scale인가?
* Q3 : model image의 learning을 어떤 식으로 진행시킬 수 있을까?
    * STDP

## Class 선언

### Cell class
* neuron의 기본적인 position, 구성요소 등을 정의하는 함수
* 후술할 ExcitatoryCell과 InhibitoryCell에서 define되는 _setup_morphology()와 _setup_biophysics() 를 __init__에서 recall함으로써 neuron을 define함

### ExcitatoryCell
* 흥분성 뉴런을 정의하는 class
    * 휴지기 전압 : -65mV
    * 초기화 전압 : -65mV
    * Threshold : -52mV
    * Refractory period : 5ms
    
### InhibitoryCell
* 억제성 뉴런을 정의하는 class
    * 휴지기 전압 : -60mV
    * 초기화 전압 : -45mV
    * Threshold : -40mV
    * Refractory period : 2ms
    
* Refractory period는 설정 완료
    * 어떻게 휴지기 전압, 초기화 전압, Threshold를 설정 가능한가?
        * 휴지기 전압, 초기화 전압의 차이점은?

In [1]:
from neuron import h, gui
from neuron.units import ms, mV
h.load_file('stdrun.hoc')

1.0

In [2]:
from neuron import h
class Cell:
    def __init__(self, gid, x, y, z):
        self._gid = gid
        self._setup_morphology()
        self.all = self.soma.wholetree()
        self._setup_biophysics()
        self.x = self.y = self.z = 0
        h.define_shape()
        self._set_position(x, y, z)
        self._spike_detector = h.NetCon(self.soma(0.5)._ref_v, None, sec=self.soma)
        self.spike_times = h.Vector()
        self._spike_detector.record(self.spike_times)
        self._ncs = []
        self.soma_v = h.Vector().record(self.soma(0.5)._ref_v)

    def __repr__(self):
        return '{}[{}]'.format(self.name, self._gid)
    
    def _set_position(self, x, y, z):
        for sec in self.all:
            for i in range(sec.n3d()):
                sec.pt3dchange(i, x - self.x + sec.x3d(i), y - self.y + sec.y3d(i), z - self.z + sec.z3d(i), sec.diam3d(i))
        self.x, self.y, self.z = x, y, z

In [3]:
class ExcitatoryCell(Cell):
    name = 'ExcitatoryCell'
    
    def _setup_morphology(self):
        self.soma = h.Section(name='soma', cell=self)
        self.dend = h.Section(name='dend', cell=self)
        self.dend.connect(self.soma)
        self.soma.L = self.soma.diam = 12.6157
        self.dend.L = 200
        self.dend.diam = 1

    def _setup_biophysics(self):
        for sec in self.all:
            sec.Ra = 100    # Axial resistance in Ohm * cm
            sec.cm = 1      # Membrane capacitance in micro Farads / cm^2
        self.soma.insert('hh')                                          
        for seg in self.soma:
            seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
            seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2
            seg.hh.gl = 0.0003    # Leak conductance in S/cm2
            seg.hh.el = -54.3     # Reversal potential in mV
            
        self.dend.insert('pas')                 
        for seg in self.dend:
            seg.pas.g = 0.001  # Passive conductance in S/cm2
            seg.pas.e = -65    # Leak reversal potential mV
        self.syn = h.ExpSyn(self.dend(0.5))
        self.syn.tau = 2 * ms
        self.refrac = 5 * ms
        # input이 .syn이라는 한 지역을 통해 입력 되는 것으로 간주시킴.
"""
h.ExpSyn decay에 의해 두 개의 ExpSyn object가 같은 point에 있는 것이나 
linearly하게 더해지는 서로 다른 두 synapse가 한 군데에 input하는 것과 다름이 없다.
이 point를 dend(0.5)로 정하자.
"""
pass

In [4]:
class InhibitoryCell(Cell):
    name = 'InhibitoryCell'
    
    def _setup_morphology(self):
        self.soma = h.Section(name='soma', cell=self)
        self.dend = h.Section(name='dend', cell=self)
        self.dend.connect(self.soma)
        self.soma.L = self.soma.diam = 12.6157
        self.dend.L = 200
        self.dend.diam = 1

    def _setup_biophysics(self):
        for sec in self.all:
            sec.Ra = 100    # Axial resistance in Ohm * cm
            sec.cm = 1      # Membrane capacitance in micro Farads / cm^2
        self.soma.insert('hh')                                          
        for seg in self.soma:
            seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
            seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2
            seg.hh.gl = 0.0003    # Leak conductance in S/cm2
            seg.hh.el = -54.3     # Reversal potential in mV
            
        self.dend.insert('pas')                 
        for seg in self.dend:
            seg.pas.g = 0.001  # Passive conductance in S/cm2
            seg.pas.e = -60    # Leak reversal potential mV
        self.syn = h.ExpSyn(self.dend(0.5))
        self.syn.tau = 2 * ms
        self.refrac = 2*ms
        # input이 .syn이라는 한 지역을 통해 입력 되는 것으로 간주시킴.
"""
h.ExpSyn decay에 의해 두 개의 ExpSyn object가 같은 point에 있는 것이나 
linearly하게 더해지는 서로 다른 두 synapse가 한 군데에 input하는 것과 다름이 없다.
이 point를 dend(0.5)로 정하자.
"""
pass

In [5]:
class ExSquare:
    def __init__(self, N=5, stim_w=0.04, stim_t=9, stim_delay=1, syn_w=0.01, syn_delay=5, d=25):
        self._syn_w = syn_w  #Stimulus weight
        self._syn_delay = syn_delay
        self._create_cells(N, d)
        self._netstim = h.NetStim()
        self._netstim.number = 1
        self._netstim.start = stim_t
        self._nc = h.NetCon(self._netstim, self.cells[0].syn)
        self._nc.delay = stim_delay
        self._nc.weight[0] = stim_w
        
    def _create_cells(self, N, d):
        self.cells = []
        for i in range(N):
            for j in range(N):
                self.cells.append(ExcitatoryCell(100*i+j, d * i, d * j, 0))
    """
    Ball And Stick #2의 create_n_BallAndStick 함수와 같은 방식으로 세포를 선언/배치.
    단, 세포들이 return되는 create_n_BallAndStick과 달리 self.cells에 저장됨.
    """
    def _connect_cells(self, insquare):
        for source, target in zip(self.cells, insquare.cells):
            nc = h.NetCon(source.soma(0.5)._ref_v, target.syn, sec=source.soma)
            nc.weight[0] = self._syn_w
            nc.delay = self._syn_delay
            source._ncs.append(nc)

In [6]:
class InSquare:
    def __init__(self, N=5, stim_w=0.04, stim_t=9, stim_delay=1, syn_w=-0.01, syn_delay=5, d=25):
        self._syn_w = syn_w  #Stimulus weight
        self._syn_delay = syn_delay
        self._create_cells(N, d)
        self._netstim = h.NetStim()
        self._netstim.number = 1
        self._netstim.start = stim_t
        self._nc = h.NetCon(self._netstim, self.cells[0].syn)
        self._nc.delay = stim_delay
        self._nc.weight[0] = stim_w
        
    def _create_cells(self, N, d):
        self.cells = []
        for i in range(N):
            for j in range(N):
                self.cells.append(InhibitoryCell(-100*i-j, d * i, d * j, -50))
    """
    Ball And Stick #2의 create_n_BallAndStick 함수와 같은 방식으로 세포를 선언/배치.
    단, 세포들이 return되는 create_n_BallAndStick과 달리 self.cells에 저장됨.
    """
    def _connect_cells(self, exsquare):
        
        for cell in self.cells:
            for excell in exsquare.cells:
                if self.cells.index(cell) != exsquare.cells.index(excell):
                    source = cell
                    target = excell
                    nc = h.NetCon(source.soma(0.5)._ref_v, target.syn, sec=source.soma)
                    nc.weight[0] = self._syn_w
                    nc.delay = self._syn_delay
                    source._ncs.append(nc)

In [7]:
exsquare = ExSquare(N=10)
insquare = InSquare(N=10)
exsquare._connect_cells(insquare)
insquare._connect_cells(exsquare)

In [8]:
print(len(exsquare.cells[0]._ncs))
print(len(insquare.cells[0]._ncs))


1
99


In [9]:
shape_window = h.PlotShape(True)
shape_window.show(0)

1.0

In [15]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

mnist = tf.keras.datasets.mnist
#x: n만큼의 random한 mnist 이미지 데이터(n, 784 차원 벡터)(n=2이면 2개의 숫자이미지 & 숫자 하나하나가 784차원)
#y: n만큼의 random한 mnist 이미지 데이터의 label(n, 10 차원 벡터)(n=2이면 2개의 숫자이미지 & 10: 라벨 개수 0~9)
x,y = mnist.train.next_batch(n)

#x(n,284) 차원의 벡터 -> (n, 28, 28) 차원의 벡터로 reshape (이미지화 시키기 위해)
mnist_image = np.array(x).reshape((28,28))
plt.title("label : " + str(np.where(y[0] == 1)[0][0]))
plt.imshow(mnist_image, cmap="gray")
plt.show()

AttributeError: module 'keras.api._v2.keras.datasets.mnist' has no attribute 'train'

In [1]:
# coding=utf-8
# Copyright 2022 The TensorFlow Datasets Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""MNIST, Fashion MNIST, KMNIST and EMNIST."""

import os
import numpy as np
from six.moves import urllib
import tensorflow as tf

import tensorflow_datasets.public_api as tfds

# MNIST constants
# CVDF mirror of http://yann.lecun.com/exdb/mnist/
_MNIST_URL = "https://storage.googleapis.com/cvdf-datasets/mnist/"
_MNIST_TRAIN_DATA_FILENAME = "train-images-idx3-ubyte.gz"
_MNIST_TRAIN_LABELS_FILENAME = "train-labels-idx1-ubyte.gz"
_MNIST_TEST_DATA_FILENAME = "t10k-images-idx3-ubyte.gz"
_MNIST_TEST_LABELS_FILENAME = "t10k-labels-idx1-ubyte.gz"
_MNIST_IMAGE_SIZE = 28
MNIST_IMAGE_SHAPE = (_MNIST_IMAGE_SIZE, _MNIST_IMAGE_SIZE, 1)
MNIST_NUM_CLASSES = 10
_TRAIN_EXAMPLES = 60000
_TEST_EXAMPLES = 10000

_MNIST_CITATION = """\
@article{lecun2010mnist,
  title={MNIST handwritten digit database},
  author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
  journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
  volume={2},
  year={2010}
}
"""

_FASHION_MNIST_CITATION = """\
@article{DBLP:journals/corr/abs-1708-07747,
  author    = {Han Xiao and
               Kashif Rasul and
               Roland Vollgraf},
  title     = {Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning
               Algorithms},
  journal   = {CoRR},
  volume    = {abs/1708.07747},
  year      = {2017},
  url       = {http://arxiv.org/abs/1708.07747},
  archivePrefix = {arXiv},
  eprint    = {1708.07747},
  timestamp = {Mon, 13 Aug 2018 16:47:27 +0200},
  biburl    = {https://dblp.org/rec/bib/journals/corr/abs-1708-07747},
  bibsource = {dblp computer science bibliography, https://dblp.org}
}
"""

_K_MNIST_CITATION = """\
  @online{clanuwat2018deep,
  author       = {Tarin Clanuwat and Mikel Bober-Irizar and Asanobu Kitamoto and Alex Lamb and Kazuaki Yamamoto and David Ha},
  title        = {Deep Learning for Classical Japanese Literature},
  date         = {2018-12-03},
  year         = {2018},
  eprintclass  = {cs.CV},
  eprinttype   = {arXiv},
  eprint       = {cs.CV/1812.01718},
}
"""

_EMNIST_CITATION = """\
@article{cohen_afshar_tapson_schaik_2017,
    title={EMNIST: Extending MNIST to handwritten letters},
    DOI={10.1109/ijcnn.2017.7966217},
    journal={2017 International Joint Conference on Neural Networks (IJCNN)},
    author={Cohen, Gregory and Afshar, Saeed and Tapson, Jonathan and Schaik, Andre Van},
    year={2017}
}
"""


class MNIST(tfds.core.GeneratorBasedBuilder):
  """MNIST."""
  URL = _MNIST_URL

  VERSION = tfds.core.Version("3.0.1")

  def _info(self):
    return tfds.core.DatasetInfo(
        builder=self,
        description=("The MNIST database of handwritten digits."),
        features=tfds.features.FeaturesDict({
            "image": tfds.features.Image(shape=MNIST_IMAGE_SHAPE),
            "label": tfds.features.ClassLabel(num_classes=MNIST_NUM_CLASSES),
        }),
        supervised_keys=("image", "label"),
        homepage="http://yann.lecun.com/exdb/mnist/",
        citation=_MNIST_CITATION,
    )

  def _split_generators(self, dl_manager):
    """Returns SplitGenerators."""
    # Download the full MNIST Database
    filenames = {
        "train_data": _MNIST_TRAIN_DATA_FILENAME,
        "train_labels": _MNIST_TRAIN_LABELS_FILENAME,
        "test_data": _MNIST_TEST_DATA_FILENAME,
        "test_labels": _MNIST_TEST_LABELS_FILENAME,
    }
    mnist_files = dl_manager.download_and_extract(
        {k: urllib.parse.urljoin(self.URL, v) for k, v in filenames.items()})

    # MNIST provides TRAIN and TEST splits, not a VALIDATION split, so we only
    # write the TRAIN and TEST splits to disk.
    return [
        tfds.core.SplitGenerator(
            name=tfds.Split.TRAIN,
            gen_kwargs=dict(
                num_examples=_TRAIN_EXAMPLES,
                data_path=mnist_files["train_data"],
                label_path=mnist_files["train_labels"],
            )),
        tfds.core.SplitGenerator(
            name=tfds.Split.TEST,
            gen_kwargs=dict(
                num_examples=_TEST_EXAMPLES,
                data_path=mnist_files["test_data"],
                label_path=mnist_files["test_labels"],
            )),
    ]

  def _generate_examples(self, num_examples, data_path, label_path):
    """Generate MNIST examples as dicts.

    Args:
      num_examples (int): The number of example.
      data_path (str): Path to the data files
      label_path (str): Path to the labels

    Yields:
      Generator yielding the next examples
    """
    images = _extract_mnist_images(data_path, num_examples)
    labels = _extract_mnist_labels(label_path, num_examples)
    data = list(zip(images, labels))

    # Using index as key since data is always loaded in same order.
    for index, (image, label) in enumerate(data):
      record = {"image": image, "label": label}
      yield index, record


class FashionMNIST(MNIST):
  URL = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/"

  # TODO(afrozm): Try to inherit from MNIST's _info and mutate things as needed.
  def _info(self):
    return tfds.core.DatasetInfo(
        builder=self,
        description=("Fashion-MNIST is a dataset of Zalando's article images "
                     "consisting of a training set of 60,000 examples and a "
                     "test set of 10,000 examples. Each example is a 28x28 "
                     "grayscale image, associated with a label from 10 "
                     "classes."),
        features=tfds.features.FeaturesDict({
            "image":
                tfds.features.Image(shape=MNIST_IMAGE_SHAPE),
            "label":
                tfds.features.ClassLabel(names=[
                    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
                    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
                ]),
        }),
        supervised_keys=("image", "label"),
        homepage="https://github.com/zalandoresearch/fashion-mnist",
        citation=_FASHION_MNIST_CITATION,
    )


class KMNIST(MNIST):
  URL = "http://codh.rois.ac.jp/kmnist/dataset/kmnist/"

  def _info(self):
    return tfds.core.DatasetInfo(
        builder=self,
        description=("Kuzushiji-MNIST is a drop-in replacement for the MNIST "
                     "dataset (28x28 grayscale, 70,000 images), provided in "
                     "the original MNIST format as well as a NumPy format. "
                     "Since MNIST restricts us to 10 classes, we chose one "
                     "character to represent each of the 10 rows of Hiragana "
                     "when creating Kuzushiji-MNIST."),
        features=tfds.features.FeaturesDict({
            "image":
                tfds.features.Image(shape=MNIST_IMAGE_SHAPE),
            "label":
                tfds.features.ClassLabel(names=[
                    "o", "ki", "su", "tsu", "na", "ha", "ma", "ya", "re", "wo"
                ]),
        }),
        supervised_keys=("image", "label"),
        homepage="http://codh.rois.ac.jp/kmnist/index.html.en",
        citation=_K_MNIST_CITATION,
    )


class EMNISTConfig(tfds.core.BuilderConfig):
  """BuilderConfig for EMNIST CONFIG."""

  def __init__(self, *, class_number, train_examples, test_examples, **kwargs):
    """BuilderConfig for EMNIST class number.

    Args:
      class_number: There are six different splits provided in this dataset. And
        have different class numbers.
      train_examples: number of train examples
      test_examples: number of test examples
      **kwargs: keyword arguments forwarded to super.
    """
    super(EMNISTConfig, self).__init__(**kwargs)
    self.class_number = class_number
    self.train_examples = train_examples
    self.test_examples = test_examples


class EMNIST(MNIST):
  """Emnist dataset."""
  URL = "https://www.itl.nist.gov/iaui/vip/cs_links/EMNIST/gzip.zip"
  VERSION = tfds.core.Version("3.0.0")
  RELEASE_NOTES = {
      "3.0.0": "New split API (https://tensorflow.org/datasets/splits)",
  }
  BUILDER_CONFIGS = [
      EMNISTConfig(
          name="byclass",
          class_number=62,
          train_examples=697932,
          test_examples=116323,
          description="EMNIST ByClass",
      ),
      EMNISTConfig(
          name="bymerge",
          class_number=47,
          train_examples=697932,
          test_examples=116323,
          description="EMNIST ByMerge",
      ),
      EMNISTConfig(
          name="balanced",
          class_number=47,
          train_examples=112800,
          test_examples=18800,
          description="EMNIST Balanced",
      ),
      EMNISTConfig(
          name="letters",
          class_number=37,
          train_examples=88800,
          test_examples=14800,
          description="EMNIST Letters",
      ),
      EMNISTConfig(
          name="digits",
          class_number=10,
          train_examples=240000,
          test_examples=40000,
          description="EMNIST Digits",
      ),
      EMNISTConfig(
          name="mnist",
          class_number=10,
          train_examples=60000,
          test_examples=10000,
          description="EMNIST MNIST",
      ),
  ]

  def _info(self):
    return tfds.core.DatasetInfo(
        builder=self,
        description=(
            "The EMNIST dataset is a set of handwritten character digits "
            "derived from the NIST Special Database 19 and converted to "
            "a 28x28 pixel image format and dataset structure that directly "
            "matches the MNIST dataset.\n\n"
            "Note: Like the original EMNIST data, images provided here are "
            "inverted horizontally and rotated 90 anti-clockwise. You can use "
            "`tf.transpose` within `ds.map` to convert the images to a "
            "human-friendlier format."),
        features=tfds.features.FeaturesDict({
            "image":
                tfds.features.Image(shape=MNIST_IMAGE_SHAPE),
            "label":
                tfds.features.ClassLabel(
                    num_classes=self.builder_config.class_number),
        }),
        supervised_keys=("image", "label"),
        homepage=("https://www.nist.gov/itl/products-and-services/"
                  "emnist-dataset"),
        citation=_EMNIST_CITATION,
    )

  def _split_generators(self, dl_manager):
    filenames = {
        "train_data":
            "emnist-{}-train-images-idx3-ubyte.gz".format(
                self.builder_config.name),
        "train_labels":
            "emnist-{}-train-labels-idx1-ubyte.gz".format(
                self.builder_config.name),
        "test_data":
            "emnist-{}-test-images-idx3-ubyte.gz".format(
                self.builder_config.name),
        "test_labels":
            "emnist-{}-test-labels-idx1-ubyte.gz".format(
                self.builder_config.name),
    }

    dir_name = os.path.join(dl_manager.download_and_extract(self.URL), "gzip")
    extracted = dl_manager.extract(
        {k: os.path.join(dir_name, fname) for k, fname in filenames.items()})

    return [
        tfds.core.SplitGenerator(
            name=tfds.Split.TRAIN,
            gen_kwargs=dict(
                num_examples=self.builder_config.train_examples,
                data_path=extracted["train_data"],
                label_path=extracted["train_labels"],
            )),
        tfds.core.SplitGenerator(
            name=tfds.Split.TEST,
            gen_kwargs=dict(
                num_examples=self.builder_config.test_examples,
                data_path=extracted["test_data"],
                label_path=extracted["test_labels"],
            ))
    ]


def _extract_mnist_images(image_filepath, num_images):
  with tf.io.gfile.GFile(image_filepath, "rb") as f:
    f.read(16)  # header
    buf = f.read(_MNIST_IMAGE_SIZE * _MNIST_IMAGE_SIZE * num_images)
    data = np.frombuffer(
        buf,
        dtype=np.uint8,
    ).reshape(num_images, _MNIST_IMAGE_SIZE, _MNIST_IMAGE_SIZE, 1)
    return data


def _extract_mnist_labels(labels_filepath, num_labels):
  with tf.io.gfile.GFile(labels_filepath, "rb") as f:
    f.read(8)  # header
    buf = f.read(num_labels)
    labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
    return labels